##Question 1

Assume the size of the file.txt below is 100 GB.<br>
Is there anything wrong with the following Spark code? If so, how can you fix it?<br>

Yes, there are a couple of issues with the Spark code: <br>
- First issue, the code returns to a value at the end, which means the code belongs to a function but the code itself doesn't contain the header for function definition. The def keyword and function name must be added before the code, e.g. "def function_name():".<br>
- Second issue, the code isn't efficient enough to read the text file data and run the code. The code starts off well by mapping first (transformation) but later it is being recomputed more than once for the same data. The code should add a cache() function before calling after the first action function. This will save reading the data in memory and prevent recomputation of it. Hence, making it more efficient to run a big size file.<br>

**Suggested fix code is shown below:**<br>

In [0]:
from collections import defaultdict

def function_name():
    text_file = sc.textFile("file.txt")
    rdd = text_file.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).cache()
    counts = rdd.collect()
    key_val = defaultdict(int)
    for item in counts:
        key = item[0]
        val = item[1]
        key_val[key] += int(val)
    filtered_key_val = dict()
    for k, v in key_val.items():
        if v >= 100:
            filtered_key_val[k] = v
    return filtered_key_val


##Question 2

Write a program in pyspark that will use the following three files:
There are three files with 150,000 questions that are asked about three programming languages in Stack Overflow, java, python, and javascript. The files are shared in D2L (Assignment 1 files).

- SO-Java contains 50,000 questions from Stack Overflow that are tagged as 'java'.<br>
- SO-Python contains 50,000 questions that are tagged as 'python'.<br>
- SO-Javascript contains 50,000 questions that are tagged as 'javascript'.<br>
- The posts are collected from Stack Overflow posts table. Details about Stack Overflow posts table can be found here:<br>

In [0]:
 %sh
 pip install --upgrade pip
 pip install beautifulsoup4
 pip install lxml
 pip install html5lib
 pip install nltk
 pip install pyspellchecker

Requirement already satisfied: pip in /databricks/python3/lib/python3.8/site-packages (21.3)
Requirement already satisfied: beautifulsoup4 in /databricks/python3/lib/python3.8/site-packages (4.10.0)
Requirement already satisfied: soupsieve>1.2 in /databricks/python3/lib/python3.8/site-packages (from beautifulsoup4) (2.2.1)
Requirement already satisfied: lxml in /databricks/python3/lib/python3.8/site-packages (4.6.3)
Requirement already satisfied: html5lib in /databricks/python3/lib/python3.8/site-packages (1.1)
Requirement already satisfied: webencodings in /databricks/python3/lib/python3.8/site-packages (from html5lib) (0.5.1)
Requirement already satisfied: six>=1.9 in /databricks/python3/lib/python3.8/site-packages (from html5lib) (1.15.0)
Requirement already satisfied: nltk in /databricks/python3/lib/python3.8/site-packages (3.6.5)
Requirement already satisfied: click in /databricks/python3/lib/python3.8/site-packages (from nltk) (8.0.3)
Requirement already satisfied: regex>=2021.8.3 in /databricks/python3/lib/python3.8/site-packages (from nltk) (2021.10.8)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.8/site-packages (from nltk) (0.17.0)
Requirement already satisfied: tqdm in /databricks/python3/lib/python3.8/site-packages (from nltk) (4.62.3)
Requirement already satisfied: pyspellchecker in /databricks/python3/lib/python3.8/site-packages (0.6.2)

**1. How can we preprocess the textual contents in the files?**

a. Write a short description of how you can answer this and then write a short program to answer this question.<br>

Depending on what type of information we want to parse out from the csv files. If we focus on the questions from the users asked in Stack Overflow, I would read the file to a dataframe first. Then, filter the dataframe by questions post only. This is done by only looking at any PostTypeId equal to 1. After the first filter, I filter it once again by any values that starts with a tag in the Body column. With this information, I can analyze and process each string of question for each row, in terms of number of words per question, number of sentences, frequent word used and others.

In [0]:
import nltk
nltk.download('punkt')
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

java_file_location = "/FileStore/tables/SO_Java.csv"
js_file_location = "/FileStore/tables/SO_Javascript.csv"
py_file_location = "/FileStore/tables/SO_Python.csv"

df_java = spark.read.format('csv').option('delimiter', ',').option('header',True).load(java_file_location)
df_js = spark.read.format('csv').option('delimiter', ',').option('header',True).load(js_file_location)
df_py = spark.read.format('csv').option('delimiter', ',').option('header',True).load(py_file_location)

questions_java = df_java.where("PostTypeId == '1'").where(df_java.Body.startswith('<') | df_java.Body.startswith('"<'))
questions_js = df_js.where("PostTypeId == '1'").where(df_js.Body.startswith('<') | df_js.Body.startswith('"<'))
questions_py = df_py.where("PostTypeId == '1'").where(df_py.Body.startswith('<') | df_py.Body.startswith('"<'))

filtered_java = questions_java.select("Body").toPandas()
filtered_js = questions_js.select("Body").toPandas()
filtered_py = questions_py.select("Body").toPandas()

def html_parser(x):
    try:
        return BeautifulSoup(x, 'lxml').text
    except:
        return x
      
clean_java = filtered_java.applymap(html_parser)
clean_js = filtered_js.applymap(html_parser)
clean_py = filtered_py.applymap(html_parser)

def get_words(SO_file):
    words = []
    for i in range(len(SO_file)):
      questions = SO_file['Body'].values[i]
      sents = sent_tokenize(questions)
      for sent in sents:
        ww = word_tokenize(sent)
        for w in ww:
          words.append(w.lower())
    return words

# It takes around 3.11 minutes to run
java_words = get_words(clean_java)
js_words = get_words(clean_js)
py_words = get_words(clean_py)

# len(java_words) # 1,607,423 words
# len(js_words) # 1,564,634 words
# len(py_words) # 1,537,088 words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Unzipping tokenizers/punkt.zip.

b. Write a function to tokenize and remove stop words from each of the files.<br>

In [0]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_en = stopwords.words('english')

def without_stop(words):
    go_words = []
    for word in words:
      if word not in stop_en:
        go_words.append(word)
    return go_words

java_nostopw = without_stop(java_words)
js_nostopw = without_stop(js_words)
py_nostopw =without_stop(py_words)

# len(java_nostopw) #916,660 words
# len(js_nostopw) #884,131 words
# len(py_nostopw) #878,283 words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!

c. Write a function to remove any other noise in the text files (first define what is a noise in the texts and then write code to remove the noise).<br>

A noise in the texts is mostly any misspelled words that is represented as an unstructured text data that could potentially be corrected as a structured or semi-structured text data. The code to remove noise from text is shown below:

In [0]:
import string
from spellchecker import SpellChecker

spell = SpellChecker()

def clean_noise(word):
    if len(spell.unknown(word)) == 0:
      return spell.correction(word)
    else:
      return word
    
rdd0_java = sc.parallelize(java_nostopw)
rdd1_java = rdd0_java.filter(lambda w: w not in string.punctuation).map(lambda w: clean_noise(w))
rdd1_java.cache()
rdd1_java.count() #number of words in java: 758,479 >> It took around 15.69 min to run

Out[16]: 758479

In [0]:
rdd0_js = sc.parallelize(js_nostopw)
rdd1_js = rdd0_js.filter(lambda w: w not in string.punctuation).map(lambda w: clean_noise(w))
rdd1_js.cache()
rdd1_js.count() #number of words in js: 732,473 >> It took around 16.11 min to run

Out[17]: 732473

In [0]:
rdd0_py = sc.parallelize(py_nostopw)
rdd1_py = rdd0_py.filter(lambda w: w not in string.punctuation).map(lambda w: clean_noise(w))
rdd1_py.cache()
rdd1_py.count() #number of words in py: 719,371 >> It took around 16.65 min to run

Out[18]: 719371

**2. What are the most frequent keywords in the textual contents of each programming language?**

a. Write a short description of how you can answer this and then write a short program to answer this question.<br>

I would get the most frequent keywords asked on each programming language by counting the number of times the word have appeared in all the questions posted. This can be done by using the groupBy and count function together to get the frequency of each word in textual contents. See code below:

**Observations:** The word 'i' should have been removed during stopwords process. It worked out if the text came with an 'I' by itself but once it was tokenized from 'I'm', it seems like the stopword process didn't completely removed from the list. I have ran the code multiple times with different adjustments but to no avail. That's why I included the top 11 words for each file instead.

In [0]:
from pyspark.sql import Row

row_rdd_java = rdd1_java.map(lambda x: Row(x))
df1_java = sqlContext.createDataFrame(row_rdd_java,['java frequent word'])
cpw_java = df1_java.groupBy('java frequent word').count()
cpw_java.orderBy(['count'], ascending=[0]).show(11)

+------------------+-----+
java frequent word|count|
+------------------+-----+
 i|21135|
 java|10755|
 'm|10441|
 using|10308|
 code| 8467|
 trying| 7769|
 want| 6599|
 file| 5918|
 like| 5334|
 class| 5300|
 application| 5113|
+------------------+-----+
only showing top 11 rows

In [0]:
row_rdd_js = rdd1_js.map(lambda x: Row(x))
df1_js = sqlContext.createDataFrame(row_rdd_js,['javascript frequent word'])
cpw_js = df1_js.groupBy('javascript frequent word').count()
cpw_js.orderBy(['count'], ascending=[0]).show(11)

+------------------------+-----+
javascript frequent word|count|
+------------------------+-----+
 i|25358|
 'm|11604|
 using|10033|
 trying| 8319|
 code| 8274|
 want| 7316|
 javascript| 6993|
 page| 6347|
 like| 6270|
 n't| 5396|
 function| 5094|
+------------------------+-----+
only showing top 11 rows

In [0]:
row_rdd_py = rdd1_py.map(lambda x: Row(x))
df1_py = sqlContext.createDataFrame(row_rdd_py,['python frequent word'])
cpw_py = df1_py.groupBy('python frequent word').count()
cpw_py.orderBy(['count'], ascending=[0]).show(11)

+--------------------+-----+
python frequent word|count|
+--------------------+-----+
 i|18095|
 python|13729|
 'm|10943|
 using| 9825|
 trying| 9737|
 code| 8703|
 want| 6899|
 like| 6603|
 file| 6220|
 data| 5834|
 get| 4623|
+--------------------+-----+
only showing top 11 rows

**3. What percentage of questions in each programming language has accepted answers?**

a. Write a short description of how you can answer this and then write a short program to answer this question.<br>

I would calculate the percentage by first counting the number of non-null values in the 'AcceptedAnswerId' column. Then, divide the counted accepted answers by the total number of questions. See code below:

In [0]:
a_answered_java = questions_java.filter(questions_java.AcceptedAnswerId.isNotNull()).select("AcceptedAnswerId").count() #25,027 accepted
a_answered_js = questions_js.filter(questions_js.AcceptedAnswerId.isNotNull()).select("AcceptedAnswerId").count() #27,643 accepted
a_answered_py = questions_py.filter(questions_py.AcceptedAnswerId.isNotNull()).select("AcceptedAnswerId").count() #25,538 accepted

perc_q_java = (a_answered_java / questions_java.count()) * 100
perc_q_js = (a_answered_js / questions_js.count()) * 100
perc_q_py = (a_answered_py /questions_py.count()) * 100

print("Percentage of questions with accepted answers in Java:", round(perc_q_java, 1), "%")
print("Percentage of questions with accepted answers in Javascript:", round(perc_q_js, 1), "%")
print("Percentage of questions with accepted answers in Python:", round(perc_q_py, 1), "%")

Percentage of questions with accepted answers in Java: 50.1 %
Percentage of questions with accepted answers in Javascript: 55.3 %
Percentage of questions with accepted answers in Python: 51.1 %

**4. What types of questions are asked for each programming languages?**

Write a short description of how you can answer this and then write a short program to answer this question, e.g., we can say a question can be of four types: How (e.g., how to solve this?), What (e.g., what is a recommended way of solving this?), Why (e.g., why is my program crashing?), or Other (everything else). To check for "why" questions, you can whether the question has started with "why" word. You can apply similar rules for find "what" and “how” type of questions.<br>

For each posted question asked in StackOverFlow, I could assume that some users may ask multiple questions in the same post. For this, I have split each question per sentence and search for any type of sentence that starts asking with a "How", "What", "Why", "When" or "Where" question type. Any sentence that doesn't start with any of the question type mentioned will be categorized as others. I will use one user-defined function to categorize each sentence for each programming language. See code below:

In [0]:
def question_types(SO_text):
    how = []
    what = []
    why = []
    when = []
    where = []
    other = []
    for i in range(len(SO_text)):
      questions = SO_text['Body'].values[i]
      sents = sent_tokenize(questions)
      for sent in sents:
        if sent.lower().startswith("how"):
            how.append(sent)
        if sent.lower().startswith("what"):
            what.append(sent)
        if sent.lower().startswith("why"):
            why.append(sent)
        if sent.lower().startswith("when"):
            when .append(sent)
        if sent.lower().startswith("where"):
            where.append(sent)
        else:
            other.append(sent)
    return len(how), len(what), len(why), len(when), len(where), len(other)

  
def print_q_summary(SO_text):
    print("question starting with...")
    print("How = ", question_types(SO_text)[0], " sentences")
    print("What = ", question_types(SO_text)[1], " sentences")
    print("Why = ", question_types(SO_text)[2], " sentences")
    print("When = ", question_types(SO_text)[3], " sentences")
    print("Where = ", question_types(SO_text)[4], " sentences")
    print("Other = ", question_types(SO_text)[5], " sentences")

In [0]:
print("-----Java Question Types-----")
print_q_summary(clean_java)

-----Java Question Types-----
question starting with...
How = 2857 sentences
What = 1085 sentences
Why = 292 sentences
When = 1819 sentences
Where = 72 sentences
Other = 93611 sentences

In [0]:
print("-----Javascript Question Types-----")
print_q_summary(clean_js)

-----Javascript Question Types-----
question starting with...
How = 2978 sentences
What = 1056 sentences
Why = 236 sentences
When = 1625 sentences
Where = 48 sentences
Other = 90522 sentences

In [0]:
print("-----Python Question Types-----")
print_q_summary(clean_py)

-----Python Question Types-----
question starting with...
How = 2845 sentences
What = 938 sentences
Why = 207 sentences
When = 1385 sentences
Where = 50 sentences
Other = 89380 sentences